In [31]:
from airqo_device_monitor.format_data import get_and_format_data_for_all_channels
from datetime import datetime, timedelta

In [51]:
start_time = datetime.utcnow() - timedelta(days=1)
data = get_and_format_data_for_all_channels(start_time=start_time)

In [45]:
public_data = {key: val for key, val in data.items() if len(val) > 0}
channel_data = public_data.values()[1]
dt = datetime.strptime(channel_data[-1].created_at, '%Y-%m-%dT%H:%M:%SZ') 

In [50]:
dt < datetime.utcnow() - timedelta(minutes=5)

False

In [40]:
dt

datetime.datetime(2018, 10, 22, 0, 13, 59)

In [41]:
datetime.utcnow()

datetime.datetime(2018, 10, 22, 8, 57, 2, 664084)

In [54]:
channel_data[-1].__dict__

{'battery_voltage': u'    4.50',
 'channel_id': 324682,
 'created_at': u'2018-10-22T09:01:52Z',
 'entry_id': 82678,
 'latitude': u'1000.000000',
 'longitude': u'1000.000000',
 'pm_1': u'    0.89',
 'pm_10': u'    3.97',
 'pm_2_5': u'    1.80',
 'sample_period': u'    1.39'}

In [43]:
public_data.keys()[0]

327840

In [87]:
LOW_BATTERY_CUTOFF = 3.5
NUM_REPORTS_TO_VERIFY_SENSOR_MALFUNCTION = 10
NUM_REPORTS_TO_VERIFY_MALFUNCTION = 10
MAXIMUM_AVERAGE_SECONDS_BETWEEN_REPORTS = 2
SENSOR_PM_2_5_MIN_CUTOFF = 1.0
SENSOR_PM_2_5_MAX_CUTOFF = 1000.0
ALLOWABLE_OUTLIER_SENSOR_RATIO = 0.2

LOW_BATTERY_MALFUNCTION_COMMENT = "The device has low battery."
LOW_REPORTING_FREQUENCY_COMMENT = "The device is reporting at low frequency."
EXTREME_SENSOR_READINGS_COMMENT = "The device's sensor is reporting outlier readings, it may be obstructed."

def get_channel_malfunction_reasons(channel_data):
    malfunction_list = []
    if has_low_battery(channel_data):
        malfunction_list.append("low_battery")
    if has_low_reporting_frequency(channel_data):
        malfunction_list.append("low_reporting_frequency")
    if may_be_obstructed(channel_data):
        malfunction_list.append("obstructed_channel")
    return malfunction_list
    
    
def has_low_battery(channel_data):
    last_voltage = float(channel_data[-1].battery_voltage)
    return last_voltage < LOW_BATTERY_CUTOFF
    
    
def has_low_reporting_frequency(channel_data):
    index_to_verify = min(len(channel_data), NUM_REPORTS_TO_VERIFY_MALFUNCTION)
    report_to_verify = channel_data[-1 * index_to_verify]
    time_to_submit_reports = datetime.strptime(report_to_verify.created_at, '%Y-%m-%dT%H:%M:%SZ')
    cutoff_time = datetime.utcnow() - timedelta(seconds=MAXIMUM_AVERAGE_SECONDS_BETWEEN_REPORTS * index_to_verify)
    return time_to_submit_reports < cutoff_time

def may_be_obstructed(channel_data):
    num_points = min(NUM_REPORTS_TO_VERIFY_SENSOR_MALFUNCTION, len(channel_data))
    is_outlier = lambda pm_2_5: pm_2_5 < SENSOR_PM_2_5_MIN_CUTOFF or pm_2_5 > SENSOR_PM_2_5_MAX_CUTOFF
    extreme_reads = [entry for entry in channel_data[-1 * num_points:] if is_outlier(float(entry.pm_2_5))]
    for point in extreme_reads:
        print point.pm_2_5
    return len(extreme_reads) > num_points * ALLOWABLE_OUTLIER_SENSOR_RATIO

In [80]:
for channel_id, channel_data in public_data.items():
    channel_malfunctions = get_channel_malfunction_reasons(channel_data)
    print "Key: {} \t Malfunction Reasons: {}".format(channel_id, channel_malfunctions)

Key: 327840 	 Malfunction Reasons: ['low_battery']
Key: 324682 	 Malfunction Reasons: []
Key: 318099 	 Malfunction Reasons: []
Key: 309890 	 Malfunction Reasons: ['low_battery']
Key: 295702 	 Malfunction Reasons: []


In [92]:
from airqo_device_monitor.format_data import get_and_format_data_for_all_channels
from datetime import datetime, timedelta


def _get_channel_malfunctions(channel_data):
    """Use channel_data to get a list of malfunctions that may be occuring with a channel."""
    malfunction_list = []
    if _has_low_battery(channel_data):
        malfunction_list.append("low_battery_voltage")
    if _has_low_reporting_frequency(channel_data):
        malfunction_list.append("low_reporting_frequency")
    if _sensor_is_reporting_outliers(channel_data):
        malfunction_list.append("reporting_outliers")
    return malfunction_list


def _has_low_battery(channel_data):
    """Determine whether the channel has low battery."""
    last_voltage = float(channel_data[-1].battery_voltage)
    return last_voltage < LOW_BATTERY_CUTOFF


def _has_low_reporting_frequency(channel_data):
    """Determine whether the channel is reporting data at a lower frequency than expected."""
    index_to_verify = min(len(channel_data), NUM_REPORTS_TO_VERIFY_MALFUNCTION)
    report_to_verify = channel_data[-1 * index_to_verify]
    time_to_submit_reports = datetime.strptime(report_to_verify.created_at, '%Y-%m-%dT%H:%M:%SZ')
    cutoff_time = datetime.utcnow() - timedelta(seconds=MAXIMUM_AVERAGE_SECONDS_BETWEEN_REPORTS * index_to_verify)
    return time_to_submit_reports < cutoff_time


def _sensor_is_reporting_outliers(channel_data):
    """Determine whether the sensor is reporting points outside the reasonable range.

    Presence of outlier points may indicated an obstructed sensor.
    """
    num_points = min(NUM_REPORTS_TO_VERIFY_SENSOR_MALFUNCTION, len(channel_data))
    is_outlier = lambda pm_2_5: pm_2_5 < SENSOR_PM_2_5_MIN_CUTOFF or pm_2_5 > SENSOR_PM_2_5_MAX_CUTOFF
    extreme_reads = [entry for entry in channel_data[-1 * num_points:] if is_outlier(float(entry.pm_2_5))]
    for point in extreme_reads:
        print point.pm_2_5
    return len(extreme_reads) > num_points * ALLOWABLE_OUTLIER_SENSOR_RATIO

def get_all_channel_malfunctions():
    """Generate a list of malfunctions for all channels.

    Returns: A dict keyed by the channel id. The value is a list of potential concerns about a sensor.
    """
    malfunctions = {}
    start_time = datetime.utcnow() - timedelta(minutes=30)
    all_data = get_and_format_data_for_all_channels(start_time=start_time)
    unusable_ids = [key for key in data.keys() if len(data[key]) == 0]
    for channel_id in unusable_ids:
        malfunctions[channel_id] = ["no_data"]

    usable_data = {key: val for key, val in data.items() if len(val) > 0}
    for channel_id, channel_data in public_data.items():
        malfunctions[channel_id] = _get_channel_malfunctions(channel_data)

    return malfunctions

In [93]:
malfunctions = get_all_channel_malfunctions()

In [94]:
malfunctions

{99649: ['no_data'],
 183070: ['no_data'],
 223586: ['no_data'],
 241694: ['no_data'],
 265435: ['no_data'],
 268271: ['no_data'],
 284171: ['no_data'],
 295702: ['low_reporting_frequency'],
 309890: ['low_battery_voltage', 'low_reporting_frequency'],
 312183: ['no_data'],
 318099: ['low_reporting_frequency'],
 324682: ['low_reporting_frequency'],
 327840: ['low_battery_voltage', 'low_reporting_frequency'],
 403696: ['no_data']}

In [91]:
a = {"key": 5}
print a.key

AttributeError: 'dict' object has no attribute 'key'